In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1578605082307_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
df_no_header=spark.read.csv('s3://network-anomaly-detection-bucket/kddcup.data.corrected',inferSchema=True,header=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df=df_no_header.toDF("duration", "protocol_type", "service", "flag", "src_bytes", "dst_bytes", "land", "wrong_fragment", "urgent",
"hot", "num_failed_logins", "logged_in", "num_compromised", "root_shell", "su_attempted", "num_root", "num_file_creations",
"num_shells", "num_access_files", "num_outbound_cmds", "is_host_login", "is_guest_login", "count", "srv_count",
"serror_rate", "srv_serror_rate", "rerror_rate", "srv_rerror_rate", "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate",
"dst_host_count", "dst_host_srv_count", "dst_host_same_srv_rate", "dst_host_diff_srv_rate", "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate",
"dst_host_serror_rate", "dst_host_srv_serror_rate", "dst_host_rerror_rate", "dst_host_srv_rerror_rate", "label")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[duration: int, protocol_type: string, service: string, flag: string, src_bytes: int, dst_bytes: int, land: int, wrong_fragment: int, urgent: int, hot: int, num_failed_logins: int, logged_in: int, num_compromised: int, root_shell: int, su_attempted: int, num_root: int, num_file_creations: int, num_shells: int, num_access_files: int, num_outbound_cmds: int, is_host_login: int, is_guest_login: int, count: int, srv_count: int, serror_rate: double, srv_serror_rate: double, rerror_rate: double, srv_rerror_rate: double, same_srv_rate: double, diff_srv_rate: double, srv_diff_host_rate: double, dst_host_count: int, dst_host_srv_count: int, dst_host_same_srv_rate: double, dst_host_diff_srv_rate: double, dst_host_same_src_port_rate: double, dst_host_srv_diff_host_rate: double, dst_host_serror_rate: double, dst_host_srv_serror_rate: double, dst_host_rerror_rate: double, dst_host_srv_rerror_rate: double, label: string]

## Silhouette coefficient

In [5]:
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.clustering import KMeans, KMeansModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import OneHotEncoder, StringIndexer, StandardScaler
from random import randint

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# StringIndexer indexes the strings to numeric values like 0, 1, 2...
# OneHotEncoder encodes the values into a vector
def oneHotPipeline(inputCol: str) -> (Pipeline, str):
    indexer = StringIndexer(inputCol=inputCol, outputCol=inputCol + '_indexed')
    encoder = OneHotEncoder(inputCol=inputCol + '_indexed', outputCol=inputCol + '_vec')
    pipeline = Pipeline(stages=[indexer, encoder])
    return (pipeline, inputCol + '_vec')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
def fitPipeline(data, k: int) -> PipelineModel:
    (protoTypeEncoder, protoTypeVecCol) = oneHotPipeline('protocol_type')
    (serviceEncoder, serviceVecCol) = oneHotPipeline('service')
    (flagEncoder, flagVecCol) = oneHotPipeline('flag')
    
    assembleCols = set(data.columns).difference({'label', 'protocol_type', 'service', 'flag'}).union({protoTypeVecCol, serviceVecCol, flagVecCol})
    
    assembler = VectorAssembler(inputCols=list(assembleCols), outputCol='featureVector')
    scaler = StandardScaler(inputCol='featureVector', outputCol='scaledFeatureVector', withStd=True, withMean=False)
    kmeans = KMeans(predictionCol='cluster', featuresCol='scaledFeatureVector', maxIter=40, tol=0.00001).setSeed(randint(1, 10)).setK(k)
    pipeline = Pipeline(stages=[protoTypeEncoder, serviceEncoder, flagEncoder, assembler, scaler, kmeans])
    pipelineModel = pipeline.fit(data)
    return pipelineModel

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# ClusteringEvaluator uses Silhouette coefficient and it's more sophisticated metric for picking k since it
# also evaluates closeness of points to other clusters
from pyspark.ml.evaluation import ClusteringEvaluator
evaluator = ClusteringEvaluator(predictionCol='cluster', featuresCol='scaledFeatureVector')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# saving the best fit model and transformed df based on silhouette score to save time
bestClustering = 'None'
bestPipelineModel = 'None'
bestScore = 0
for k in range(60, 271, 30):
    pipelineModel = fitPipeline(df, k)
    clustered = pipelineModel.transform(df)
    score = evaluator.evaluate(clustered)
    if (score >= bestScore):
        bestScore = score
        bestClustering = clustered 
        bestPipelineModel = pipelineModel
    print(k, score)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

60 0.5574937678949499
90 0.7152617625802837
120 0.9438389992866768
150 0.9311556636332504
180 0.7689924927275976
210 0.8682422149852521
240 0.7459220131478589
270 0.769550308252886

In [22]:
# Using 120 as the value for k
countByClusterLabel = bestClustering.select('cluster', 'label').groupBy('cluster', 'label').count().orderBy('cluster', 'label')
countByClusterLabel.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+------+
|cluster|     label| count|
+-------+----------+------+
|      0|     back.|  1441|
|      0|   normal.|251846|
|      0|portsweep.|     1|
|      1|  neptune.|  1041|
|      1|portsweep.|     9|
|      1|    satan.|     2|
|      2|  neptune.|  1038|
|      2|portsweep.|     9|
|      2|    satan.|     3|
|      3|  neptune.|822500|
|      3|portsweep.|   142|
|      4|  ipsweep.|    13|
|      4|  neptune.|  1039|
|      4|   normal.|     7|
|      4|portsweep.|    11|
|      4|    satan.|     3|
|      5|  neptune.|  1040|
|      5|portsweep.|    13|
|      5|    satan.|     3|
|      6|portsweep.|     2|
+-------+----------+------+
only showing top 20 rows

In [23]:
from pyspark.ml.linalg import Vector, Vectors
kMeansModel = bestPipelineModel.stages[-1]
centroids = kMeansModel.clusterCenters()
vectorDistances = bestClustering.select('cluster', 'scaledFeatureVector').rdd.map(
lambda row: Vectors.squared_distance(centroids[row.cluster], row.scaledFeatureVector)).collect()
vectorDistances.sort(reverse=True) # so we can see the resuls in DESC order
threshold = vectorDistances[:100][-1] # 100th-farthest point distance as a threshold

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
threshold

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8688.4043852374

In [25]:
# Using silhouette score to find the best k value (in this case 120) provides a greater threshold value considering it's the
# 100th-farthest point distance
# This makes sense since we have less clusters which translates into bigger distances from the centroids of each cluster to the
# farthest points of each cluster (easier to understand with a drawing)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# let's see how many anomalies are detected

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
len(bestClustering.rdd.filter(
    lambda row: Vectors.squared_distance(centroids[row.cluster], row.scaledFeatureVector) >= threshold).collect())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

100

In [28]:
# same anomalies detected as k=210 and distance 4623.4134839215185
# silhouette coefficient allowed to pick a more efficient value of k since it provides the same result with less computation

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Gaussian mixture

In [ ]:
from pyspark.ml.clustering import GaussianMixture

In [ ]:
def fitGaussianMixturePipeline(data, k: int) -> PipelineModel:
    (protoTypeEncoder, protoTypeVecCol) = oneHotPipeline('protocol_type')
    (serviceEncoder, serviceVecCol) = oneHotPipeline('service')
    (flagEncoder, flagVecCol) = oneHotPipeline('flag')
    
    assembleCols = set(data.columns).difference({'label', 'protocol_type', 'service', 'flag'}).union({protoTypeVecCol, serviceVecCol, flagVecCol})
    
    assembler = VectorAssembler(inputCols=list(assembleCols), outputCol='featureVector')
    scaler = StandardScaler(inputCol='featureVector', outputCol='scaledFeatureVector', withStd=True, withMean=False)
    gmm = GaussianMixture(predictionCol='cluster', featuresCol='scaledFeatureVector', maxIter=40, tol=0.00001).setSeed(randint(1, 10)).setK(k)
    pipeline = Pipeline(stages=[protoTypeEncoder, serviceEncoder, flagEncoder, assembler, scaler, gmm])
    pipelineModel = pipeline.fit(data)
    return pipelineModel

In [31]:
# we can also use the previously defined evaluator to evaluate the clustering made using Gaussian Mixture Model
# only using a 1% sample since Gaussian Mixture Model is very slow!

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df1 = df.sample(0.01)

In [ ]:
pipelineModel = fitGaussianMixturePipeline(df1, 120)

In [ ]:
predictions = pipelineModel.transform(df1)

In [ ]:
score = evaluator.evaluate(predictions)

In [22]:
score

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.5024759721658255

In [17]:
# bad score. Only using a sample and one value of k but it's 
# the worst score even considering all scores returned by the evaluator for the multiple values of k in kmeans clustering
# very slow operation, takes hours just to fit with a sample. would need more time and more processing power to test 
# multiple k values and test with the whole dataset. could take multiple days

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Bisecting KMeans

In [29]:
from pyspark.ml.clustering import BisectingKMeans

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
def fitBisectingKMeansPipeline(data, k: int) -> PipelineModel:
    (protoTypeEncoder, protoTypeVecCol) = oneHotPipeline('protocol_type')
    (serviceEncoder, serviceVecCol) = oneHotPipeline('service')
    (flagEncoder, flagVecCol) = oneHotPipeline('flag')
    
    assembleCols = set(data.columns).difference({'label', 'protocol_type', 'service', 'flag'}).union({protoTypeVecCol, serviceVecCol, flagVecCol})
    
    assembler = VectorAssembler(inputCols=list(assembleCols), outputCol='featureVector')
    scaler = StandardScaler(inputCol='featureVector', outputCol='scaledFeatureVector', withStd=True, withMean=False)
    bisectingKMeans = BisectingKMeans(predictionCol='cluster', featuresCol='scaledFeatureVector', maxIter=40).setSeed(randint(1, 10)).setK(k)
    pipeline = Pipeline(stages=[protoTypeEncoder, serviceEncoder, flagEncoder, assembler, scaler, bisectingKMeans])
    pipelineModel = pipeline.fit(data)
    return pipelineModel

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
pipelineModel = fitBisectingKMeansPipeline(df, 120)

In [ ]:
predictions = pipelineModel.transform(df)

In [ ]:
score = evaluator.evaluate(predictions)

In [23]:
score

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.4841816043291794